In [1]:
import debugger_magic

In [ ]:
%%debugger_magic
print(1234)


def _add_two(v):
    return v + 2


def _mul_two(v):
    out = v * 2.0
    return out


str1 = 'foo'
str2 = '你好'
str3 = str1 + str2
x = 1
y = 11
z = x + y
if z > 10:
    for i in range(5):
        z = z * 2
    z = _add_two(z ** 2)
    z = _mul_two(z)
print(z)

In [2]:
%%debugger_magic
import tensorflow as tf
tf.enable_eager_execution()



def _add_one(v):
    return v + 1


def _mul_two(v):
    out = v * 2.0
    return out


def main():
#   a = tf.constant([[1, 2], [30, 40]], dtype=tf.float32)
    all_as = tf.random_normal([10, 20, 20], dtype=tf.float32)
    a = tf.unstack(all_as, axis=0)[0]
    b = tf.random_uniform(
        [20, 1], dtype=tf.float32)
    c = tf.matmul(a, b)
    x = 8
    y = 9
    v = [1, 2, 3, 4, 5, 6, 7]  # Array
    z = tf.constant(x + y, dtype=tf.float32)
    # Invoke _add_one()
    z = _add_one(z)
    z = _mul_two(z)
    print(z)

    
main()

<IPython.core.display.Javascript object>

In [ ]:
import asyncio
async def fn():
  print('hello')
  await asyncio.sleep(1)
  print('world')

await fn()

In [ ]:
raise KeyboardInterrupt

In [ ]:
_i2